# Route Montag 
### (Vormittag) 
<br>
Since the author recommends to visit surrounding cities in the afternoon (which are not on my map), I decided to limit my analysis to the late morning. <br>
But of course, those would suffice as interesting sources as well. 

In [51]:
from os import path
xml_path = path.relpath("../Data/pages_8_47.xml")

from bs4 import BeautifulSoup   
with open(xml_path, encoding='utf8') as file:   
    contents = file.read()
    soup = BeautifulSoup(contents, 'xml')

In [52]:
import re

start_tag = 'hi'   
start_string = 'Montag.'    
end_tag = 'hi'     
end_string = 'Nachmittag'  
place_tag = 'placeName'     
should_start_parsing = False   

In [53]:
Montag = []        #here, the whole text will be placed - in case we need it later on
Places = []     #here, the places for our route will be stored
for line in soup.find_all('l'):
    if line.string == start_string and line.string.parent.name == start_tag:
        should_start_parsing = True  

    if should_start_parsing:
        text = line.get_text()
        Montag.append(text)
        for child in line.descendants:
            if child.name == place_tag:
                for text in child.strings:
                    Places.append(text)          

    if line.string == end_string and line.string.parent.name == end_tag:
        should_start_parsing = False  
        break

In [58]:
while(" " in Places):
    Places.remove(" ")
print(Places) 

clean_places = []
for e in Places:
    if clean_places and clean_places[-1].endswith('='):
        clean_places[-1] = clean_places[-1][:-1] + e
    elif clean_places and (clean_places[-1].endswith('des ') or clean_places[-1].endswith('des')):
        clean_places[-1] = clean_places[-1] + ' ' + e
    elif clean_places and (clean_places[-1].endswith('der ') or clean_places[-1].endswith('der')): 
        clean_places[-1] = clean_places[-1] + ' ' + e     
    elif clean_places and clean_places[-1].endswith('den '): 
        clean_places[-1] = clean_places[-1] + ' ' + e 
    elif clean_places and (clean_places[-1].endswith('Fürsten ') or clean_places[-1].endswith('.')):
        clean_places[-1] = clean_places[-1] + ' ' + e         
    else:
        clean_places.append(e)


['kaiserlichen', 'Zeughauses', 'Renngasse Nr. 140', 'Renngasse', 'hohe Brücke', 'tiefen Graben', 'Schwert=', 'gasse', 'Maria Stiegen', 'Salzgries', 'Maria am Gestade', 'Stoß im Himmel', 'Wipplingerstraße', 'Rathhaus', 'Hofkanzlei', 'Quergasse', 'Judenplatz', 'den Hof.', 'Kirche zu den ', '7 Chören der Engel', 'bürgerliche Zeughaus', 'Bognergasse', 'Seitzerhof', 'Spenglergasse', 'Milchgäßchen', 'Petersplatz', 'Peterskirche', 'Polizei=', 'Oberdirektion', 'Feuerwache', 'Goldschmiedgasse', 'Stephansplatz', 'Gerolds', ' Buchhandlung', 'Stephansdom', 'Stephansplatze', 'Bischofgasse', 'Wollzeile', 'Schranders ', 'Kleidermagazin', 'Brief=', 'Post', 'Schaumburgs eleganter Bücher=', 'laden', 'Essiggäßchen', 'Bäckerstraße', 'Universitätsplatz', 'Universitäts=Gebäude', 'Sternwarte', 'Konvikt', 'Uni=', 'versitätshaus', 'Schulgasse', 'Dominkanerplatz', 'Universitätsbibllothek', 'Fahrpost', 'grie=', 'chische Kirche', 'Hauptmauth', 'Hôtel zur Stadt London', 'Klosternenburg', 'Sommer=', 'palais', 'Gart

In [59]:
print(clean_places)
print(len(clean_places))
# not correct, should end after "Hotel zur Stadt London"

['kaiserlichen', 'Zeughauses', 'Renngasse Nr. 140', 'Renngasse', 'hohe Brücke', 'tiefen Graben', 'Schwertgasse', 'Maria Stiegen', 'Salzgries', 'Maria am Gestade', 'Stoß im Himmel', 'Wipplingerstraße', 'Rathhaus', 'Hofkanzlei', 'Quergasse', 'Judenplatz', 'den Hof. Kirche zu den  7 Chören der Engel', 'bürgerliche Zeughaus', 'Bognergasse', 'Seitzerhof', 'Spenglergasse', 'Milchgäßchen', 'Petersplatz', 'Peterskirche', 'PolizeiOberdirektion', 'Feuerwache', 'Goldschmiedgasse', 'Stephansplatz', 'Gerolds', ' Buchhandlung', 'Stephansdom', 'Stephansplatze', 'Bischofgasse', 'Wollzeile', 'Schranders ', 'Kleidermagazin', 'BriefPost', 'Schaumburgs eleganter Bücherladen', 'Essiggäßchen', 'Bäckerstraße', 'Universitätsplatz', 'Universitäts=Gebäude', 'Sternwarte', 'Konvikt', 'Universitätshaus', 'Schulgasse', 'Dominkanerplatz', 'Universitätsbibllothek', 'Fahrpost', 'griechische Kirche', 'Hauptmauth', 'Hôtel zur Stadt London', 'Klosternenburg', 'Sommerpalais', 'Garten des  Fürsten  Liechtenstein', 'Rossau'

Something in my code doesn't work - it should cut after "Nachmittag", but doesn't do that on my machine. <br>
Because I couldn't find the mistake, I decided to leave the code and just cut my list manually. <br>
In return, I was able to add a little snippet in the end of the Monday-chapter, where the entrance back to the city as well as the location of the orphanage is described. 

In [60]:
#split depending on indices
i = clean_places.index('Hôtel zur Stadt London')
j = clean_places.index('Tulnerhof') #that is the last place before returning to the city 

del clean_places[i+1:j+1] # delete entrys from the afternoon (from the last place of morning until the last place before city entry) 

k = clean_places.index('Waisenhaus') #that is the last place of Monday
del clean_places[k+1:] # delete the redundant places 
clean_places

['kaiserlichen',
 'Zeughauses',
 'Renngasse Nr. 140',
 'Renngasse',
 'hohe Brücke',
 'tiefen Graben',
 'Schwertgasse',
 'Maria Stiegen',
 'Salzgries',
 'Maria am Gestade',
 'Stoß im Himmel',
 'Wipplingerstraße',
 'Rathhaus',
 'Hofkanzlei',
 'Quergasse',
 'Judenplatz',
 'den Hof. Kirche zu den  7 Chören der Engel',
 'bürgerliche Zeughaus',
 'Bognergasse',
 'Seitzerhof',
 'Spenglergasse',
 'Milchgäßchen',
 'Petersplatz',
 'Peterskirche',
 'PolizeiOberdirektion',
 'Feuerwache',
 'Goldschmiedgasse',
 'Stephansplatz',
 'Gerolds',
 ' Buchhandlung',
 'Stephansdom',
 'Stephansplatze',
 'Bischofgasse',
 'Wollzeile',
 'Schranders ',
 'Kleidermagazin',
 'BriefPost',
 'Schaumburgs eleganter Bücherladen',
 'Essiggäßchen',
 'Bäckerstraße',
 'Universitätsplatz',
 'Universitäts=Gebäude',
 'Sternwarte',
 'Konvikt',
 'Universitätshaus',
 'Schulgasse',
 'Dominkanerplatz',
 'Universitätsbibllothek',
 'Fahrpost',
 'griechische Kirche',
 'Hauptmauth',
 'Hôtel zur Stadt London',
 'Währingergasse',
 'Palais D

In [61]:
#manual refinement: combine 'kaiserlich' and 'Zeughauses' as well as 'Gerolds' & 'Buchhandlung'
import functools

clean_places[0:2] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[0:2])]
clean_places[27:29] = [functools.reduce(lambda x, y: x + ' ' + y, clean_places[27:29])]

In [62]:
#again: compare words to compile a list with only one version of varying place_names in the list

import difflib as dl

Similiar_words = []
for word in clean_places:
    close_words = dl.get_close_matches(word, clean_places, cutoff= 0.9)     #find matches of similar written places
    Similiar_words.append(close_words)                                      #gather them in a list

place_list = []
for sublist in Similiar_words:     #only the first word or version of every similar-places-combination gets chosen as 'standard'
    sublist = sorted(sublist)
    place = sublist[0]
    place_list.append(place)  

place_list_text= open('../Data/monday_place_list_text.txt','w', encoding='utf8')      #save the new list of places in a file
for item in place_list:
	place_list_text.write(item+"\n")
place_list_text.close()      

Let's start building our coordinate-table! <br>
As before, We will start with the df from Wien Geschichte Wiki, add the coordinates from the Gazetteer and finally compare it to the manually built dataframe! 

In [63]:
import pandas as pd
csv_path = path.relpath("../Data/KULTURWIKIOGD.csv")    #import the downloaded dataset
usecolumns = ['OBJECTID', 'SHAPE', 'ADRESSE', 'SEITENNAME']    #since the dataset is pretty big, I decided to use only some of the columns for efficieny
coordinates = pd.read_csv(csv_path,  index_col="OBJECTID", usecols=usecolumns)

from thefuzz import process, fuzz 

coordinates['MATCH_SCORE'] = coordinates['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1]) 
coordinates['NEUE_NAMEN'] = coordinates['SEITENNAME'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])
#  use uniform names for our places &
# create a new column called MATCH_SCORE, where the resulting scores of the comparison between our place_list will be stored. 

coordinates = coordinates.sort_values('MATCH_SCORE', ascending=False)
coordinates_choice = coordinates.drop(coordinates[coordinates['MATCH_SCORE'] <= 84].index) #I used 84 again as threshold, since it fit pretty well

#  use uniform names for our places 

coordinates_choice = coordinates_choice.drop_duplicates(subset=['NEUE_NAMEN']) 
place_coord  = coordinates_choice.drop(columns = ['SEITENNAME', 'MATCH_SCORE']) #drop unnceccessary columns

place_coord = place_coord.set_index('NEUE_NAMEN')  #set new index
place_coord = place_coord.reindex(index=place_list) #re-index again: corresponding records get ordered according to the place list
place_coord = place_coord.reset_index() #reset the index
place_coord.reset_index(inplace=True)
place_coord = place_coord.drop(columns=['index'])
place_coord.to_csv('../Data/Montag_coord.csv', sep=',', index=False, encoding='utf-8') #save our new Dataframe of places

In [64]:
place_coord

,NEUE_NAMEN,SHAPE,ADRESSE
0,kaiserlichen Zeughauses,NaN,NaN
1,Renngasse Nr. 140,NaN,NaN
2,Renngasse,POINT (16.367211551082715 48.21277258486223),"01., Renngasse 12"
3,hohe Brücke,POINT (16.368607210105097 48.21273703123713),"01., Hohe Brücke 120"
4,tiefen Graben,POINT (16.36762957026213 48.212022628343945),"01., Tiefer Graben 12"
5,Schwertgasse,POINT (16.36968259868288 48.21263236404844),"01., Schwertgasse 2"
6,Maria Stiegen,NaN,NaN
7,Salzgries,POINT (16.372856189480476 48.212974028083316),"01., Salztorgasse 2"
8,Maria am Gestade,POINT (16.371131768915312 48.21243145165297),"01., Salvatorgasse 12/1"
9,Stoß im Himmel,POINT (16.37023622465546 48.212049341689465),"01., Stoß im Himmel 1"


Again, let's also try the Gazetteer: 

In [65]:
xlsx_path = path.relpath("../Data/Gazetteer_Steinhausenplan_V5.xlsx")
usecolumns = ['ID_neu', 'Toponym', 'Sicherheit', 'Longitude', 'Latitude']
Steinhausen_coordinates = pd.read_excel(xlsx_path, usecols=usecolumns, index_col=0)  

Steinhausen_coordinates['MATCH_SCORE'] = Steinhausen_coordinates['Toponym'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[1])
Steinhausen_coordinates['NEUE_NAMEN'] = Steinhausen_coordinates['Toponym'].apply(lambda x: process.extractOne(x, place_list, scorer=fuzz.ratio)[0])
Steinhausen_coordinates = Steinhausen_coordinates.sort_values('MATCH_SCORE', ascending=False)

#print(Steinhausen_coordinates.head(50))
Steinhausen_coordinates = Steinhausen_coordinates.drop(Steinhausen_coordinates[Steinhausen_coordinates['MATCH_SCORE'] <= 77].index)
Steinhausen_choice = Steinhausen_coordinates.drop_duplicates(subset=['NEUE_NAMEN'])

Steinhausen_choice = Steinhausen_choice.set_index('NEUE_NAMEN')
Steinhausen_choice = Steinhausen_choice.reindex(index=place_list)
Steinhausen_choice = Steinhausen_choice.reset_index()
Steinhausen_choice.reset_index(inplace=True)

Steinhausen_choice = Steinhausen_choice.drop(columns=['Sicherheit', 'index', 'MATCH_SCORE', 'Toponym'])
Steinhausen_choice.to_csv('../Data/Steinhausen_Montag_coord.csv', sep=',', index=False, encoding='utf-8')

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query

In [66]:
#split shape into long & lat
place_coord['SHAPE'] = place_coord['SHAPE'].str.replace(r'POINT', '', regex=True)
place_coord['SHAPE'] = place_coord['SHAPE'].str.replace(r'\)', '', regex=True)
place_coord[['Longitude', 'Latitude']] = place_coord['SHAPE'].str.extract('(\d+.\d+)\s(\d+.\d+)', expand=True)

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sarah\AppData\Local\Temp\ipykernel_9428\2634734394.py:4: SyntaxWarning: invalid escape sequence '\d'
  place_coord[['Longitude', 'Latitude']] = place_coord['SHAPE'].str.extract('(\d+.\d+)\s(\d+.\d+)', expand=True)


In [126]:
joined_places = pd.merge(place_coord, Steinhausen_choice, left_index=True, right_index=True)
joined_places = joined_places.drop(columns=["NEUE_NAMEN_y", "SHAPE"])

We should compare our dataframe with our already established dataframe from sunday to be able to fill some gaps:

In [71]:
Sunday_gold_table = pd.read_excel("../Data/joined_Sonntag_coord_nn.xlsx", index_col=0) 

In [72]:
Sunday_gold_table

,Index,NEUE_NAMEN,ADRESSE,Sicherheit,Longitude,Latitude,Quellen,URL,Long_GoogleMaps,Lat_ GoogleMaps
0,0,k. k. Burg,NaN,hoch,16.365809,48.206874,Gazetteer_Steinhausenplan_V5_auto,NaN,NaN,NaN
1,1,Burgplatz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Burgthor,NaN,niedrig,16.363591,48.207307,Gazetteer_Steinhausenplan_V5_auto,NaN,NaN,NaN
3,3,kaiserlichen Stallungen,NaN,NaN,16.376115,48.207519,Gazetteer_Steinhausenplan_V5_manuell,NaN,NaN,NaN
4,4,Gebäude der ungarischen Garde,"07., Museumstraße 7",NaN,16.355278,48.205833,"Wikipedia, GeoHack",https://geohack.toolforge.org/geohack.php?page...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
183,183,Volksgarten,"01., Volksgarten THESEUSTEMPEL",NaN,16.361755,48.207964,NaN,NaN,NaN,NaN
184,184,Theseustempel,"01., Volksgarten THESEUSTEMPEL",NaN,16.361686,48.208383,NaN,NaN,NaN,NaN
185,185,Hof=Burgwache,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,186,Schweizerhof,"01., Hofburg B Schweizertrakt",NaN,16.365843,48.206876,NaN,NaN,NaN,NaN


In [103]:
matches = joined_places.merge(Sunday_gold_table, left_on='NEUE_NAMEN_x', right_on='NEUE_NAMEN').drop('NEUE_NAMEN', axis='columns')

In [106]:
matches

,NEUE_NAMEN_x,ADRESSE_x,Longitude_x,Latitude_x,Longitude_y,Latitude_y,Index,ADRESSE_y,Sicherheit,Longitude,Latitude,Quellen,URL,Long_GoogleMaps,Lat_ GoogleMaps
0,Salzgries,"01., Salztorgasse 2",16.372856189480476,48.212974028083316,16.371645,48.213246,123,"01., Salztorgasse 2",mittel,16.372856,48.212974,NaN,NaN,NaN,NaN
1,Wollzeile,"01., Wollzeile 20",16.376389522715947,48.20834155448045,16.375334,48.208713,72,"01., Wollzeile 20",NaN,16.376390,48.208342,Gazetteer_Steinhausenplan_V5_auto,NaN,NaN,NaN
2,Fahrpost,NaN,NaN,NaN,NaN,NaN,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hauptmauth,NaN,NaN,NaN,NaN,NaN,87,NaN,NaN,16.379060,48.210195,NaN,NaN,NaN,NaN
4,Hauptmauth,NaN,NaN,NaN,NaN,NaN,89,"03., Dominikanerbastei 9-13",NaN,16.379060,48.210195,NaN,NaN,NaN,NaN
5,Währingergasse,"04., Weyringergasse 24",16.37660993287555,48.18851950935964,16.358229,48.218601,151,NaN,hoch,16.358229,48.218601,Gazetteer_Steinhausenplan_V5_auto,NaN,NaN,NaN


As we see above, there are some mutual places. Apparently we found some coordinates for the Hauptmauth in the Sunday-dataframe - that's good to know! <br>
I'm guessing (from my order of columns in the Sunday df) that those column-vlaues are from the Wien Geschichte Wiki. <br>
In that case, we can transfer them into our Monday-df without hesitation. <br>
(otherwise, manual transfer might have been wiser)

In [133]:
import numpy as np
joined_places['Latitude_x'] = np.where(joined_places['NEUE_NAMEN_x'] == 'Hauptmauth', 48.210195	, joined_places['Latitude_x'])
joined_places['Longitude_x'] = np.where(joined_places['NEUE_NAMEN_x'] == 'Hauptmauth', 16.379060, joined_places['Longitude_x'])

In [134]:
joined_places

,NEUE_NAMEN_x,ADRESSE,Longitude_x,Latitude_x,Longitude_y,Latitude_y
0,kaiserlichen Zeughauses,NaN,NaN,NaN,NaN,NaN
1,Renngasse Nr. 140,NaN,NaN,NaN,NaN,NaN
2,Renngasse,"01., Renngasse 12",16.367211551082715,48.21277258486223,16.367072,48.212571
3,hohe Brücke,"01., Hohe Brücke 120",16.368607210105097,48.21273703123713,NaN,NaN
4,tiefen Graben,"01., Tiefer Graben 12",16.36762957026213,48.212022628343945,16.368608,48.212734
5,Schwertgasse,"01., Schwertgasse 2",16.36968259868288,48.21263236404844,NaN,NaN
6,Maria Stiegen,NaN,NaN,NaN,NaN,NaN
7,Salzgries,"01., Salztorgasse 2",16.372856189480476,48.212974028083316,16.371645,48.213246
8,Maria am Gestade,"01., Salvatorgasse 12/1",16.371131768915312,48.21243145165297,NaN,NaN
9,Stoß im Himmel,"01., Stoß im Himmel 1",16.37023622465546,48.212049341689465,NaN,NaN


In [135]:
joined_places.to_excel('../Data/joined_Montag_coord.xlsx')